#### Reference List

1. [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
2. 밑바닥부터 시작하는 딥러닝 2
3. [Keras 내 LSTM 구현체](https://github.com/keras-team/keras/blob/master/keras/layers/recurrent.py#L1766)


╔══<i><b>Alai-DeepLearning</b></i>════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 13. RNN-Basis**
# Section 5. LSTM 계층 조립하기

### _Objective_

1. LSTM은 기존의 RNN과 달리, 상태(state)와 기억(Memory)을 나누어 관리하는 것이 특징입니다.<br>
2. 이 모델은 RNNCell의 핵심 문제인 장기의존 관계를 효과적으로 해결했습니다. <br>
3. LSTM은 현대 딥러닝 모델에 있어서, 기본적으로 쓰이는 RNN Cell로 자리매김하였습니다.


╚═════════════════════════════════════════╝

In [1]:
from tensorflow.keras.layers import Layer
import tensorflow as tf

/Users/ksj/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


<br><br>

# \[ 1. LSTM이란? \]
---
---

> *RNN의 핵심 이슈 중 하나인 BPTT(Back Propagation Through Time)는 Input과 output 사이의 긴 Time Step이 존재할 때, 학습을 어렵게 합니다.*<br>
> *이러한 이슈를 통칭하여 장기기억 이슈라 하여, Time Step 간의 길이가 길 때 어떻게 효과적으로 학습시킬지가 관건입니다.*<br>
> *RNN의 Cell을 개선하여 이를 효과적으로 해결한 것이 바로 LSTM입니다.*<br>

<br>

## 1. RNN 복습하기, RNNCell

----

* 아래는 이전 시간에 배웠던 RNNCell의 기본 형태입니다. RNN은 RNNCell을 time step 별로 반복하여 연산하는 구조를 가지고 있습니다.

![Imgur](https://imgur.com/RNEHbhZ.png)


In [2]:
class RNNCell(Layer):
    """
    Keras에서 Cell을 구성하는 Layer
    
    1. __init__ : Cell에 필요한 hyper-parameter를 구성
       상태 벡터의 크기를 의미하는 self.state_size는 필수적으로 정해주어야 함
       마지막에 super([Layer],self).__init__(**kwargs)를 호출해야함

    2. build : Cell에서 관리하는 Weight들을 선언. 
       마지막에 super([Layer],self).build()를 호출해야함
    
    3. call : Cell의 연산 순서를 정의
    
    """
    def __init__(self, n_units, **kwargs):
        self.n_units = n_units
        self.state_size = n_units
        super(RNNCell, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.w_xh = self.add_weight(name='weight_xh',
                                    shape=(input_shape[-1], self.n_units),
                                    initializer='glorot_normal')
        self.w_hh = self.add_weight(name='weight_hh',
                                    shape=(self.n_units, self.n_units),
                                    initializer='orthogonal')
        self.b_h = self.add_weight(name='bias_h',
                                   shape=(self.n_units,),
                                   initializer='zeros')
        super(RNNCell, self).build(input_shape)
        
    def call(self, inputs, states):
        prev_states = states[0]
        h = (tf.matmul(inputs, self.w_xh) 
             + tf.matmul(prev_states, self.w_hh)
             + self.b_h)
        a = tf.tanh(h)
        return a, [a]

입력값과 출력값 사이의 간격(time steps)가 길다면, 어떤 문제가 발생할까요? 중요한 입력값은 state을 가지고 전파되는데 여기서 weight들이 계속 적용되면서 중요한 입력값은 점점 그 값을 소실하는 문제가 발생합니다. 이러한 문제는 time step의 길이가 10이나 20정도만 되어도 기존의 RNN은 경사하강법으로 훈련해서 학습에 성공할 확률이 매우 낮아지게 됩니다.<br>


<br>

## 2. LSTM의 인터페이스 

---

![Imgur](https://imgur.com/4m5fgb9.png)

* LSTM Cell에는 기존 RNN과 달리 기억 셀(Memory Cell)이 존재하며, LSTM 전용의 기억 메커니즘을 가지고 있습니다.
* LSTM 계층의 인터페이스에는 C라는 기억 셀이 존재합니다. 이것은 LSTM 계층 사이에서의 전용 `기억` 정보로, 자기 자신으로만 주고받는 것입니다. 

In [3]:
class LSTMCell(Layer):
    def __init__(self, n_units, **kwargs):
        self.n_units = n_units
        # 두 개의 State (C,H)를 가지기 때문에 
        # self.state_size는 (n_units, n_units)가 됩니다.
        self.state_size = (n_units, n_units)
        super(LSTMCell, self).__init__(**kwargs)

<br>

## 3. LSTM 내 Gate

---

![Imgur](https://imgur.com/iAKgtRs.png)

* Sigmoid 함수로 이루어진 Gate는 0~1 범위의 출력값을 가지고 있습니다. Gate는 데이터의 흐름을 제어합니다. 데이터가 흘러가지 못하게 할 때에는 Gate의 값이 0으로 줄이고, 데이터가 흐르게 할때에는 Gate의 값을 1로 만듭니다.

* LSTM에는 총 3개의 Gate가 존재합니다. 하나는 출력의 데이터를 제어하는 Output GATE, 그리고 기억을 지우는 Forget GATE, 마지막으로 기억을 더하는 Input GATE로 나뉘어집니다. 우리는 이것을 바탕으로 하나씩 구성해보도록 하겠습니다.

### (1) LSTM의 Output GATE

![Imgur](https://imgur.com/k7uvjsb.png)

* 기억 정보에는 시각 t에서의 LSTM의 기억이 저장되어 있습니다. 이러한 기억을 바탕으로, 외부 계층으로 빠져나가는 은닉 상태 $h_t$를 결정합니다. 이 때 출력하는 $h_t$는 아래와 같이 기억 셀의 값($c_t$)을 tanh 함수로 변환한 값입니다.
* 단순히 기억의 값을 출력하는 것이 아니라, output GATE는 "**기억의 요소 중 어떤 것들을 출력할까**"를 결정하게 됩니다. 현재 상태와 이전 기억을 종합해서 값을 출력하게 됩니다.

    $
    {gate}^{(o)} = \sigma(x_t\cdot W_x^{(o)}+ h_t\cdot W_h^{(o)}+b^{(o)}) \\
    output = {gate}^{(o)} \odot tanh(c_t)
    $
    

* 수식에 존재하는 $\odot$은 아마다르곱이라고 부르는 연산자로, 같은 크기의 두 행렬의 각 성분을 곱하는 연산입니다. Numpy 연산 중 `*`를 떠올리면 됩니다.

In [4]:
class LSTMCell(Layer):
    def __init__(self, n_units, **kwargs):
        self.n_units = n_units
        self.state_size = (n_units, n_units)
        super(LSTMCell, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # output gate에 관련된 weight 선언
        self.wx_output = self.add_weight("weight_x_output",
                                         shape=(input_shape[-1],self.n_units),
                                         initializer='glorot_uniform')
        self.wh_output = self.add_weight('weight_h_output',
                                         shape=(self.n_units,self.n_units),
                                         initializer='glorot_uniform')        
        self.b_output = self.add_weight('bias_output',
                                        shape=(self.n_units,),
                                        initializer='zeros')
        super(LSTMCell, self).build(input_shape)
        
    def call(self, x, states):
        h, c = states
        
        # output에 관련된 처리들
        output_gate = tf.sigmoid(tf.dot(x,self.wx_output)+
                                 tf.dot(h,self.wh_output)+
                                 self.b_output)
        new_h = output_gate * tf.tanh(c)
        return new_h, [new_h, c]
    

### (2) LSTM의 Forget GATE

![Imgur](https://imgur.com/HxrGcW5.png)

* 기억해야 할 것과 기억하지 말아야할 것을 분리해야 합니다. 기억 셀에 있는 것 중 현재 상태로 미루어보아, 불 필요한 부분들은 제거할 필요가 있습니다.
* forget Gate는 기억 셀에서 무엇을 잊을까를 명확하게 지시하는 것입니다.

    $
    {gate}^{(f)} = \sigma(x_t\cdot W_x^{(f)}+ h_t\cdot W_h^{(f)}+b^{(f)}) \\
    c'_{t-1} = {gate}^{(f)} \odot c_{t-1}
    $

In [19]:
class LSTMCell(Layer):
    def __init__(self, n_units, **kwargs):
        self.n_units = n_units
        self.state_size = (n_units, n_units)
        super(LSTMCell, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Forget Gate에 관련된 weight 선언
        self.wx_forget = self.add_weight("weight_x_forget",
                                         shape=(input_shape[-1],self.n_units),
                                         initializer='glorot_uniform')
        self.wh_forget = self.add_weight('weight_h_forget',
                                         shape=(self.n_units,self.n_units),
                                         initializer='orthogonal')
        self.b_forget = self.add_weight('bias_forget',
                                        shape=(self.n_units,),
                                        initializer='zeros')
        
        # output gate에 관련된 weight 선언
        self.wx_output = self.add_weight("weight_x_output",
                                         shape=(input_shape[-1],self.n_units),
                                         initializer='glorot_uniform')
        self.wh_output = self.add_weight('weight_h_output',
                                         shape=(self.n_units,self.n_units),
                                         initializer='orthogonal')
        self.b_output = self.add_weight('bias_output',
                                        shape=(self.n_units,),
                                        initializer='zeros')
        super(LSTMCell, self).build(input_shape)
        
    def call(self, x, states):
        h, c = states
        # forget에 관련된 처리들
        forget_gate = tf.sigmoid(tf.dot(x,self.wx_forget)+
                                 tf.dot(h,self.wh_forget)+
                                 self.b_forget)
        c = forget_gate * c        
        
        # output에 관련된 처리들
        output_gate = tf.sigmoid(tf.dot(x,self.wx_output)+
                                 tf.dot(h,self.wh_output)+
                                 self.b_output)
        new_h = output_gate * tf.tanh(c)
        return new_h, [new_h, c]
    

### (3) LSTM의 Input GATE

![Imgur](https://imgur.com/g9iI4We.png)

* 마지막으로 새로 기억해야 할 정보를 기억 셀에 추가하는 작업이 필요합니다.<br>
* 그리고 기억할 요소들의 가치 판단은 input 게이트에서 합니다. 단순히 모든 기억할 요소들을 저장하는 것이 아닌, input Gate에서 필요한 것만 취사선택하도록 학습합니다.<br>
* 즉 갱신할 정보를 만드는 `tanh` 부분과 갱신할 정보의 수준을 조절하는 `sigmoid` 부분으로 나누어 볼 수 있습니다.

    $
    update = tanh(x_t\cdot W_x^{(u)}+ h_t\cdot W_h^{(u)}+b^{(u)})\\
    gate^{(i)} = \sigma(x_t\cdot W_x^{(i)}+ h_t\cdot W_h^{(i)}+b^{(i)}) \\
    c_t = c'_{t-1} + update \odot {gate}^{(i)}
    $

In [ ]:
class LSTMCell(Layer):
    def __init__(self, n_units, **kwargs):
        self.n_units = n_units
        self.state_size = (n_units, n_units)
        super(LSTMCell, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Forget Gate에 관련된 weight 선언
        self.wx_forget = self.add_weight("weight_x_forget",
                                         shape=(input_shape[-1],self.n_units),
                                         initializer='glorot_uniform')
        self.wh_forget = self.add_weight('weight_h_forget',
                                         shape=(self.n_units,self.n_units),
                                         initializer='orthogonal')
        self.b_forget = self.add_weight('bias_forget',
                                        shape=(self.n_units,),
                                        initializer='zeros')
        # update에 관련된 weight 선언
        self.wx_update = self.add_weight("weight_x_update",
                                         shape=(input_shape[-1],self.n_units),
                                         initializer='glorot_uniform')
        self.wh_update = self.add_weight('weight_h_update',
                                         shape=(self.n_units,self.n_units),
                                         initializer='orthogonal')
        self.b_update = self.add_weight('bias_update',
                                        shape=(self.n_units,),
                                        initializer='zeros')
        # Input Gate에 관련된 Weight 선언
        self.wx_input = self.add_weight("weight_x_input",
                                         shape=(input_shape[-1],self.n_units),
                                         initializer='glorot_uniform')
        self.wh_input = self.add_weight('weight_h_input',
                                         shape=(self.n_units,self.n_units),
                                         initializer='orthogonal')
        self.b_input = self.add_weight('bias_input',
                                        shape=(self.n_units,),
                                        initializer='zeros')
        # output gate에 관련된 weight 선언
        self.wx_output = self.add_weight("weight_x_output",
                                         shape=(input_shape[-1],self.n_units),
                                         initializer='glorot_uniform')
        self.wh_output = self.add_weight('weight_h_output',
                                         shape=(self.n_units,self.n_units),
                                         initializer='orthogonal')
        self.b_output = self.add_weight('bias_output',
                                        shape=(self.n_units,),
                                        initializer='zeros')
        super(LSTMCell, self).build(input_shape)
        
    def call(self, x, states):
        h, c = states
        # forget에 관련된 처리들
        forget_gate = tf.sigmoid(tf.dot(x,self.wx_forget)+
                                 tf.dot(h,self.wh_forget)+
                                 self.b_forget)
        # update에 관련된 처리들
        update = tf.tanh(tf.dot(x,self.wx_update)+
                         tf.dot(h,self.wh_update)+
                         self.b_update)
        input_gate = tf.sigmoid(tf.dot(x,self.wx_input)+
                                tf.dot(h,self.wh_input)+
                                self.b_input)
        new_c = forget_gate * c + update * input_gate
        # output에 관련된 처리들
        output_gate = tf.sigmoid(tf.dot(x,self.wx_output)+
                                 tf.dot(h,self.wh_output)+
                                 self.b_output)
        new_h = output_gate * tf.tanh(new_c)
        return new_h, [new_h, new_c]

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/06/18

---